<a href="https://colab.research.google.com/gist/qbeer/c7630c11339b659843e32e39eb732e42/hw5_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Logistic regression

### 1. Download data from

https://science.sciencemag.org/content/359/6378/926 (supplementary materials).

* read the abstract of the article to get familiar with data origin
* open the data in excel and get familiar with its content
* load the protein level data (you need to figure out which one is that) as a pandas dataframe
* handle missing values and convert features to numeric values when it is needed
* get rid of the unnecessary (which does not encode protein levels or the tumor type) columns and the CancerSEEK results



In [143]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys 
import os
import conda

import seaborn as sns

#toc
from nbindex import floating
floating.tableofcontent()

ModuleNotFoundError: No module named 'nbindex'

In [144]:
#the needed data is Table S6
#there are 2 rows that we dont need
data0 = pd.read_excel('data/aar3247_cohen_sm_tables-s1-s11.xlsx', sheet_name='Table S6', skiprows=2)
display(data0.head())

,Patient ID #,Sample ID #,Tumor type,AJCC Stage,AFP (pg/ml),Angiopoietin-2 (pg/ml),AXL (pg/ml),CA-125 (U/ml),CA 15-3 (U/ml),CA19-9 (U/ml),...,sFas (pg/ml),SHBG (nM),sHER2/sEGFR2/sErbB2 (pg/ml),sPECAM-1 (pg/ml),TGFa (pg/ml),Thrombospondin-2 (pg/ml),TIMP-1 (pg/ml),TIMP-2 (pg/ml),CancerSEEK Logistic Regression Score,CancerSEEK Test Result
0,CRC 455,CRC 455 PLS 1,Colorectum,I,1583.45,5598.5,3621.04,5.09,19.08,*16.452,...,*204.792,55.06,6832.07,9368.53,*16.086,21863.74,56428.71,39498.82,0.938342,Positive
1,CRC 456,CRC 456 PLS 1,Colorectum,I,*715.308,20936.35,2772.96,7.27,10.04,40.91,...,*204.792,72.92,5549.47,6224.55,*16.086,29669.66,73940.49,41277.09,0.925363,Positive
2,CRC 457,CRC 457 PLS 1,Colorectum,II,4365.53,2350.93,4120.77,*4.854,16.96,*16.452,...,*204.792,173.78,3698.16,4046.48,179.03,6020.47,22797.28,28440.60,0.852367,Negative
3,CRC 458,CRC 458 PLS 1,Colorectum,II,*715.308,1604.34,2029.96,5.39,8.31,*16.452,...,*204.792,29.47,5856,6121.93,*16.086,4331.02,20441.19,25896.73,0.617639,Negative
4,CRC 459,CRC 459 PLS 1,Colorectum,II,801.3,2087.57,2069.17,*4.854,11.73,*16.452,...,*204.792,78.07,5447.93,6982.32,*16.086,2311.91,56288.51,49425.20,0.318434,Negative


These stars! They use it to flag some discrepancy, or for different usage. Do they cause problem?

In [145]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 45 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Patient ID #                          1820 non-null   object 
 1   Sample ID #                           1817 non-null   object 
 2   Tumor type                            1817 non-null   object 
 3   AJCC Stage                            1005 non-null   object 
 4   AFP (pg/ml)                           1817 non-null   object 
 5   Angiopoietin-2 (pg/ml)                1817 non-null   object 
 6   AXL (pg/ml)                           1811 non-null   float64
 7   CA-125 (U/ml)                         1817 non-null   object 
 8   CA 15-3 (U/ml)                        1817 non-null   object 
 9   CA19-9 (U/ml)                         1817 non-null   object 
 10  CD44 (ng/ml)                          1811 non-null   object 
 11  CEA (pg/ml)      

Almost all columns have one or two starred numbers! 

In [146]:
import regex as re

In [147]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
data0 = data0[:-4].replace(to_replace=r'[\*,\*\*]', value='', regex=True)
display(data0)
#facepalm, yes, this that simple... now 'to_numeric'

,Patient ID #,Sample ID #,Tumor type,AJCC Stage,AFP (pg/ml),Angiopoietin-2 (pg/ml),AXL (pg/ml),CA-125 (U/ml),CA 15-3 (U/ml),CA19-9 (U/ml),...,sFas (pg/ml),SHBG (nM),sHER2/sEGFR2/sErbB2 (pg/ml),sPECAM-1 (pg/ml),TGFa (pg/ml),Thrombospondin-2 (pg/ml),TIMP-1 (pg/ml),TIMP-2 (pg/ml),CancerSEEK Logistic Regression Score,CancerSEEK Test Result
0,CRC 455,CRC 455 PLS 1,Colorectum,I,1583.45,5598.5,3621.04,5.09,19.08,16.452,...,204.792,55.06,6832.07,9368.53,16.086,21863.74,56428.71,39498.82,0.938342,Positive
1,CRC 456,CRC 456 PLS 1,Colorectum,I,715.308,20936.35,2772.96,7.27,10.04,40.91,...,204.792,72.92,5549.47,6224.55,16.086,29669.66,73940.49,41277.09,0.925363,Positive
2,CRC 457,CRC 457 PLS 1,Colorectum,II,4365.53,2350.93,4120.77,4.854,16.96,16.452,...,204.792,173.78,3698.16,4046.48,179.03,6020.47,22797.28,28440.60,0.852367,Negative
3,CRC 458,CRC 458 PLS 1,Colorectum,II,715.308,1604.34,2029.96,5.39,8.31,16.452,...,204.792,29.47,5856,6121.93,16.086,4331.02,20441.19,25896.73,0.617639,Negative
4,CRC 459,CRC 459 PLS 1,Colorectum,II,801.3,2087.57,2069.17,4.854,11.73,16.452,...,204.792,78.07,5447.93,6982.32,16.086,2311.91,56288.51,49425.20,0.318434,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812,PAPA 1353,PAPA 1353 PLS 1,Ovary,I,879.498,1484.7,2096.76,24.82,10.3,42.39,...,207.24,115.24,5390.31,8538.58,16.89,599.4,167799.61,50128.60,0.980312,Positive
1813,PAPA 1354,PAPA 1354 PLS 1,Ovary,I,1337.33,1607.9,852.37,5.58,9.8,16.44,...,207.24,147.17,7951.03,12966.19,16.89,599.4,123443.76,54066.98,0.999995,Positive
1814,PAPA 1355,PAPA 1355 PLS 1,Ovary,III,879.498,1592.84,1044.45,30.48,8.48,16.44,...,207.24,104.63,2396.36,1901.41,16.89,599.4,104070.89,39844.02,1.000000,Positive
1815,PAPA 1356,PAPA 1356 PLS 1,Ovary,II,879.498,5267.95,1445.69,1469.45,23.74,62.26,...,207.24,73.55,3079.81,5312.90,16.89,6864.33,110579.24,42921.13,1.000000,Positive


In [148]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1817 entries, 0 to 1816
Data columns (total 45 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Patient ID #                          1817 non-null   object 
 1   Sample ID #                           1817 non-null   object 
 2   Tumor type                            1817 non-null   object 
 3   AJCC Stage                            1005 non-null   object 
 4   AFP (pg/ml)                           1817 non-null   object 
 5   Angiopoietin-2 (pg/ml)                1817 non-null   object 
 6   AXL (pg/ml)                           1811 non-null   float64
 7   CA-125 (U/ml)                         1817 non-null   object 
 8   CA 15-3 (U/ml)                        1817 non-null   object 
 9   CA19-9 (U/ml)                         1817 non-null   object 
 10  CD44 (ng/ml)                          1811 non-null   object 
 11  CEA (pg/ml)      

In [149]:
for column in data0.columns[4:-1]:
    data0[column] = pd.to_numeric(data0[column])
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1817 entries, 0 to 1816
Data columns (total 45 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Patient ID #                          1817 non-null   object 
 1   Sample ID #                           1817 non-null   object 
 2   Tumor type                            1817 non-null   object 
 3   AJCC Stage                            1005 non-null   object 
 4   AFP (pg/ml)                           1817 non-null   float64
 5   Angiopoietin-2 (pg/ml)                1817 non-null   float64
 6   AXL (pg/ml)                           1811 non-null   float64
 7   CA-125 (U/ml)                         1817 non-null   float64
 8   CA 15-3 (U/ml)                        1817 non-null   float64
 9   CA19-9 (U/ml)                         1817 non-null   float64
 10  CD44 (ng/ml)                          1811 non-null   float64
 11  CEA (pg/ml)      

In [150]:
for i in data0.columns:
    print("{0} NaNs in \t{1}".format(sum(data0[i].isna()),i))

0 NaNs in 	Patient ID #
0 NaNs in 	Sample ID #
0 NaNs in 	Tumor type
812 NaNs in 	AJCC Stage
0 NaNs in 	AFP (pg/ml)
0 NaNs in 	Angiopoietin-2 (pg/ml)
6 NaNs in 	AXL (pg/ml)
0 NaNs in 	CA-125 (U/ml)
0 NaNs in 	CA 15-3 (U/ml)
0 NaNs in 	CA19-9 (U/ml)
6 NaNs in 	CD44 (ng/ml)
0 NaNs in 	CEA (pg/ml)
0 NaNs in 	CYFRA 21-1 (pg/ml)
0 NaNs in 	DKK1 (ng/ml)
0 NaNs in 	Endoglin (pg/ml)
0 NaNs in 	FGF2 (pg/ml)
0 NaNs in 	Follistatin (pg/ml)
0 NaNs in 	Galectin-3 (ng/ml)
7 NaNs in 	G-CSF (pg/ml)
0 NaNs in 	GDF15 (ng/ml)
0 NaNs in 	HE4 (pg/ml)
0 NaNs in 	HGF (pg/ml)
0 NaNs in 	IL-6 (pg/ml)
0 NaNs in 	IL-8 (pg/ml)
6 NaNs in 	Kallikrein-6 (pg/ml)
0 NaNs in 	Leptin (pg/ml)
6 NaNs in 	Mesothelin (ng/ml)
6 NaNs in 	Midkine (pg/ml)
0 NaNs in 	Myeloperoxidase (ng/ml)
0 NaNs in 	NSE (ng/ml)
0 NaNs in 	OPG (ng/ml)
0 NaNs in 	OPN (pg/ml)
6 NaNs in 	PAR (pg/ml)
0 NaNs in 	Prolactin (pg/ml)
6 NaNs in 	sEGFR (pg/ml)
1 NaNs in 	sFas (pg/ml)
0 NaNs in 	SHBG (nM)
6 NaNs in 	sHER2/sEGFR2/sErbB2 (pg/ml)
6 NaNs in 	sP

In [151]:
nyani = pd.unique(data0['AJCC Stage'])[3]
data0['AJCC Stage'] = data0['AJCC Stage'].replace('I','1').replace('II','2').replace('III','3').replace(nyani,'0')
pd.unique(data0['AJCC Stage'])
data0 = data0.dropna()
data0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 0 to 1816
Data columns (total 45 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Patient ID #                          1803 non-null   object 
 1   Sample ID #                           1803 non-null   object 
 2   Tumor type                            1803 non-null   object 
 3   AJCC Stage                            1803 non-null   object 
 4   AFP (pg/ml)                           1803 non-null   float64
 5   Angiopoietin-2 (pg/ml)                1803 non-null   float64
 6   AXL (pg/ml)                           1803 non-null   float64
 7   CA-125 (U/ml)                         1803 non-null   float64
 8   CA 15-3 (U/ml)                        1803 non-null   float64
 9   CA19-9 (U/ml)                         1803 non-null   float64
 10  CD44 (ng/ml)                          1803 non-null   float64
 11  CEA (pg/ml)      

<font size="6"> VERI NICE, WE CLEANED IT! </font>

In [152]:
cancerSeeker_outcome = data0[data0.columns[-2:]]
display(cancerSeeker_outcome) 
tumor_type = data0[data0.columns[2]]
display(tumor_type)

,CancerSEEK Logistic Regression Score,CancerSEEK Test Result
0,0.938342,Positive
1,0.925363,Positive
2,0.852367,Negative
3,0.617639,Negative
4,0.318434,Negative
...,...,...
1812,0.980312,Positive
1813,0.999995,Positive
1814,1.000000,Positive
1815,1.000000,Positive


0       Colorectum
1       Colorectum
2       Colorectum
3       Colorectum
4       Colorectum
           ...    
1812         Ovary
1813         Ovary
1814         Ovary
1815         Ovary
1816         Ovary
Name: Tumor type, Length: 1803, dtype: object

In [153]:
data0 = data0.drop(columns = ['CancerSEEK Logistic Regression Score','CancerSEEK Test Result'])

In [154]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 0 to 1816
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Patient ID #                 1803 non-null   object 
 1   Sample ID #                  1803 non-null   object 
 2   Tumor type                   1803 non-null   object 
 3   AJCC Stage                   1803 non-null   object 
 4   AFP (pg/ml)                  1803 non-null   float64
 5   Angiopoietin-2 (pg/ml)       1803 non-null   float64
 6   AXL (pg/ml)                  1803 non-null   float64
 7   CA-125 (U/ml)                1803 non-null   float64
 8   CA 15-3 (U/ml)               1803 non-null   float64
 9   CA19-9 (U/ml)                1803 non-null   float64
 10  CD44 (ng/ml)                 1803 non-null   float64
 11  CEA (pg/ml)                  1803 non-null   float64
 12  CYFRA 21-1 (pg/ml)           1803 non-null   float64
 13  DKK1 (ng/ml)      

### 2. Predict if a sample is cancerous or not

* you need to build a classifier that predicts the probability of a sample coming from a cancerous (tumor type is normal or not) person based on the measured protein levels
* train a logistic regression (sklearn API) on every second sample (not first 50% of the data (!), use every second line)
* generate prediction for the samples that were not used during the training



In [155]:
data0[data0.columns[2]].value_counts()

Normal        799
Colorectum    388
Breast        208
Lung          104
Pancreas       93
Stomach        68
Ovary          54
Esophagus      45
Liver          44
Name: Tumor type, dtype: int64

If normal -> 0

Everything else is cancerous -> 1

In [158]:
data0[data0.columns[2]] = data0[data0.columns[2]].replace(['Normal','Colorectum','Breast','Lung', 'Pancreas', 'Stomach', 'Ovary', 'Esophagus', 'Liver'], 
                                ['0','1','1','1','1','1','1','1','1'])
data0[data0.columns[2]].value_counts()  #strange one-hot encoding hahaha!

1    1004
0     799
Name: Tumor type, dtype: int64

In [115]:
#this is for testing if i know it right!
#lista = np.array([10,20,30,40])
#display(lista[1::2])

###  3. Comparision to CancerSEEK
* plot the ROC curve and calculate the confusion matrix for the predictions
* do the same for the CancerSEEK predictions
* compare your model's performance to CancerSEEK performance



### 4. Hepatocellular carcinoma

* fit a logistic regression (using statsmodels API this time) to predict if a sample has Hepatocellular carcinoma (liver cancer) or not. You need to keep only the liver and the normal samples for this exercise! For fitting use only the first 25 features and all the rows (which are liver or normal)
* select the 5 best predictor based on P values.
* Write down the most important features (based on P value) and compare them to the tumor markers that you find on wikipeida https://en.wikipedia.org/wiki/Hepatocellular_carcinoma or other sources!



### 5. Multiclass classification

* Again, using every second datapoint train a logistic regression (sklearn API) to predict the tumor type. It is a multiclass classification problem.
* Generate prediction for the rest of the dataset and show the confution matrix for the predictions!
* Plot the ROC curves for the different cancer types on the same plot!
* Intepret your results. Which cancer type can be predicted the most reliably?



### Hints:

* On total you can get 10 points for fully completing all tasks.
* Decorate your notebook with, questions, explanation etc, make it self contained and understandable!
* Comments you code when necessary
* Write functions for repetitive tasks!
* Use the pandas package for data loading and handling
* Use matplotlib and seaborn for plotting or bokeh and plotly for interactive investigation
* Use the scikit learn package for almost everything
* Use for loops only if it is really necessary!
* Code sharing is not allowed between student! Sharing code will result in zero points.
* If you use code found on web, it is OK, but, make its source clear!